In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd

import wows_shell

with open('current_lgb.json') as f:
    lgd_shell_data = json.load(f)

gp_ship_local_ship = {}
ship_artillery_shells = {}
for __nation, nation_data in lgd_shell_data['ships'].items():
    for __type, type_data in nation_data.items():
        for gp_ship, ship_data in type_data.items():
            local_ship = ship_data['Name']
            gp_ship_local_ship[gp_ship] = local_ship

            org_artilleries = ship_data['artillery']
            final_artilleries = {}
            for artillery, artillery_data in org_artilleries.items():
                ap_shell_names = []
                for shell_type, shell_name in artillery_data['shells'].items():
                    if shell_type == 'AP':
                        ap_shell_names.append(shell_name)
                
                if len(ap_shell_names) > 0:
                    final_artilleries[artillery] = {'shells': ap_shell_names}

            ship_artillery_shells[gp_ship] = final_artilleries


ap_shells = {}
for shell, shell_data in lgd_shell_data['shells'].items():
    if shell_data['alphaPiercingHE'] == 0 and shell_data['alphaPiercingCS'] == 0:
        ap_shells[shell] = shell_data

#print(ship_artillery_shells.keys())
#print(ap_shells.keys())
print(ap_shells['PAPA001_Shell_203mm_AP_AP_Mk_19'])

#print(ship_artillery_shells['PUSB507_Yukon'])

{'alphaPiercingCS': 0, 'alphaPiercingHE': 0.0, 'bulletAirDrag': 0.321, 'bulletAlwaysRicochetAt': 67.5, 'bulletCapNormalizeMaxAngle': 7.0, 'bulletDetonator': 0.033, 'bulletDetonatorThreshold': 34.0, 'bulletDiametr': 0.203, 'bulletKrupp': 2846.0, 'bulletMass': 118.0, 'bulletRicochetAt': 60.0, 'bulletSpeed': 853.0}


In [2]:

ship_art_wows_shells = {}
calculator = wows_shell.shellCalc()

def old_penetration (krupp, mass, caliber, normalization, impact_speed, impact_angle):
    raw = krupp / 2400 * impact_speed  ** 1.4822064892953855 * mass ** 0.5506 * caliber ** -0.6521 * 0.00046905491615181766 
    angle = raw / np.cos(impact_angle)
    normalization_rad = np.deg2rad(normalization)
    normalized = raw / np.cos(np.maximum(0, np.abs(impact_angle) - normalization_rad))

    return (raw, angle, normalized)

def find_nearest_index (arr, target):
    difference = np.abs(arr - target)
    return difference.argmin()

test_distances = [0, 5, 10, 15, 20, 25]

ship_artillery_data = {}
ship_artillery_difference = {
    'ship': [],
    'artillery': [],
    'shell': [],
    'test_distance': [],
    'difference': [],
}

for ship, artilleries in ship_artillery_shells.items():
    ship_artillery_data[ship] = {}
    for artillery, artillery_data in artilleries.items():
        ship_artillery_data[ship][artillery] = []
        for shell in artillery_data['shells']:
            shell_data = ap_shells[shell]
            caliber = shell_data['bulletDiametr']
            mass = shell_data['bulletMass']
            krupp = shell_data['bulletMass']
            normalization = shell_data['bulletCapNormalizeMaxAngle']
            s = wows_shell.shell(wows_shell.shellParams(
                caliber, 
                shell_data['bulletSpeed'], 
                shell_data['bulletAirDrag'], 
                mass, 
                krupp, 
                normalization, 
                shell_data['bulletDetonator'], 
                shell_data['bulletDetonatorThreshold'], 
                shell_data['bulletRicochetAt'], 
                shell_data['bulletAlwaysRicochetAt'], 
                0), 
                '')
            
            calculator.calcImpactForwardEuler(s)
            impact_data = s.getImpact()
            distance = impact_data[int(wows_shell.impactIndices.distance), :]
            velocity = impact_data[int(wows_shell.impactIndices.impactVelocity), :]
            impact_angle = impact_data[int(wows_shell.impactIndices.impactAngleHorizontalRadians), :]

            raw, angle, normalized = old_penetration(krupp, mass, caliber, normalization, velocity, impact_angle)
            new_raw = impact_data[int(wows_shell.impactIndices.rawPenetration), :]
            new_angle = impact_data[int(wows_shell.impactIndices.effectivePenetrationHorizontal), :]
            new_normalized = impact_data[int(wows_shell.impactIndices.effectivePenetrationHorizontalNormalized), :]
            percent_difference = 100 * (new_raw / raw - 1)

            ship_artillery_data[ship][artillery].append(pd.DataFrame(data={
                'distance': distance,
                'old_raw': raw, 
                'old_angle': angle, 
                'old_normalized': normalized,
                'new_raw': new_raw, 
                'new_angle': new_angle, 
                'new_normalized': new_normalized,
                'difference': percent_difference
            }))

            for test_distance in test_distances:
                test_distance_m = test_distance * 1000
                idx = find_nearest_index(distance, test_distance_m)
                
                diff = float("NaN")
                if abs(distance[idx] - test_distance_m) < 1000:
                    diff = percent_difference[idx]
                
                '''ship_artillery_difference_row = pd.DataFrame(data={
                    'ship': [ship],
                    'artillery': [artillery],
                    'shell': [shell],
                    'test_distance': [test_distance_m],
                    'difference': [diff]
                })'''

                #ship_artillery_difference.concat(ship_artillery_difference_row, ignore_index=True)

                ship_artillery_difference['ship'].append(ship)
                ship_artillery_difference['artillery'].append(artillery)
                ship_artillery_difference['shell'].append(shell)
                ship_artillery_difference['test_distance'].append(test_distance_m)
                ship_artillery_difference['difference'].append(diff)


for k, v in ship_artillery_difference.items():
    print(f'{k} {len(v)}')

ship_artillery_difference = pd.DataFrame(ship_artillery_difference)
print(ship_artillery_difference)


ship 4794
artillery 4794
shell 4794
test_distance 4794
difference 4794
                       ship     artillery                     shell  \
0             PUSB507_Yukon   A_Artillery    PUPA014_381MM_AP_YUKON   
1             PUSB507_Yukon   A_Artillery    PUPA014_381MM_AP_YUKON   
2             PUSB507_Yukon   A_Artillery    PUPA014_381MM_AP_YUKON   
3             PUSB507_Yukon   A_Artillery    PUPA014_381MM_AP_YUKON   
4             PUSB507_Yukon   A_Artillery    PUPA014_381MM_AP_YUKON   
...                     ...           ...                       ...   
4789  PBSD605_Anthony_Event  A1_Artillery  PBPA048_120MM_QFMIX_AP_G   
4790  PBSD605_Anthony_Event  A1_Artillery  PBPA048_120MM_QFMIX_AP_G   
4791  PBSD605_Anthony_Event  A1_Artillery  PBPA048_120MM_QFMIX_AP_G   
4792  PBSD605_Anthony_Event  A1_Artillery  PBPA048_120MM_QFMIX_AP_G   
4793  PBSD605_Anthony_Event  A1_Artillery  PBPA048_120MM_QFMIX_AP_G   

      test_distance  difference  
0                 0   -0.943621  
1       

In [3]:
with open('results.csv', 'w') as f:
    ship_artillery_difference.to_csv(f)